In [1]:
import os
import sys
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [2]:
cos_credentials = {
    "apikey": "XXXX-XXXX-XXXX-XXXX",
    "cos_hmac_keys": {
        "access_key_id": "XXXX-XXXX-XXXX-XXXX",
        "secret_access_key": "XXXX-XXXX-XXXX-XXXX"
    },
    "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
    "iam_apikey_description": "XXXX-XXXX-XXXX-XXXX",
    "iam_apikey_name": "XXXX-XXXX-XXXX-XXXX",
    "iam_role_crn": "XXXX-XXXX-XXXX-XXXX",
    "iam_serviceid_crn": "XXXX-XXXX-XXXX-XXXX",
    "resource_instance_id": "XXXX-XXXX-XXXX-XXXX"
}

service_endpoint = "https://s3.private.eu-gb.cloud-object-storage.appdomain.cloud"

buckets = {
    "training": "bucket-training-gb-po",
    "results": "bucket-results-gb-po"
}

wml_credentials = {
    "instance_id": "XXXX-XXXX-XXXX-XXXX",
    "password": "XXXX-XXXX-XXXX-XXXX",
    "url": "https://eu-gb.ml.cloud.ibm.com",
    "username": "XXXX-XXXX-XXXX-XXXX"
}

In [3]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [4]:
execution_command = "echo ${DATA_DIR}; echo ${RESULT_DIR}; export PYTHONPATH=$PYTHONPATH:${DATA_DIR}/models/:${DATA_DIR}/models/research/:${DATA_DIR}/models/research/slim/; python3 export_inference_graph.py --input_type=image_tensor --pipeline_config_path=${DATA_DIR}/models/research/object_detection/training/faster_rcnn_inception_v2_coco.config --trained_checkpoint_prefix=${DATA_DIR}/models/research/object_detection/training/model.ckpt-17818 --output_directory=${RESULT_DIR}; python3 object_detection_image.py"

In [5]:
model_definition_metadata = {
    client.repository.DefinitionMetaNames.NAME: "repository-casino-2",
    client.repository.DefinitionMetaNames.DESCRIPTION: "Casino model",
    client.repository.DefinitionMetaNames.AUTHOR_NAME: "Mathis OUDIN",
    client.repository.DefinitionMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.DefinitionMetaNames.FRAMEWORK_VERSION: "1.10",
    client.repository.DefinitionMetaNames.RUNTIME_NAME: "python",
    client.repository.DefinitionMetaNames.RUNTIME_VERSION: "3.5",
    client.repository.DefinitionMetaNames.EXECUTION_COMMAND: execution_command
}

In [6]:
definition_details = client.repository.store_definition("test.zip", model_definition_metadata)
definition_uid = client.repository.get_definition_uid(definition_details)
print("The definition_uid is: {}".format(definition_uid))

The definition_uid is: da2b4b87-c86e-4562-8c14-ab772a46f98d


In [7]:
training_configuration_metadata = {
    client.training.ConfigurationMetaNames.NAME: "training-casino-2", 
    client.training.ConfigurationMetaNames.AUTHOR_NAME: "Mathis OUDIN",              
    client.training.ConfigurationMetaNames.DESCRIPTION: "Casino beer model",
    client.training.ConfigurationMetaNames.COMPUTE_CONFIGURATION: {"name": "k80"},
    client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCE: {
        "connection": {
            "endpoint_url": service_endpoint,
            "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
            "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
        },
        "source": {
            "bucket": buckets['training'],
        },
        "type": "s3"
    },
    client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {
        "connection": {
            "endpoint_url": service_endpoint,
            "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
            "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
        },
        "target": {
            "bucket": buckets['results'],
        },
        "type": "s3"
    },
}

In [8]:
training_run_details = client.training.run(definition_uid, training_configuration_metadata)
training_run_guid_async = client.training.get_run_uid(training_run_details)
print("The training_run_guid_async is: {}".format(training_run_guid_async))

The training_run_guid_async is: model-oqjsrjex


In [9]:
client.training.monitor_logs(training_run_guid_async)



####################################################

Log monitor started for training run: model-oqjsrjex

####################################################


training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 
training-Wgt3huXig: 


-----------------
Log monitor done.
-----------------




In [ ]:
saved_model_details = client.repository.store_model(training_run_guid_async, {"name": "My object detection model"})
print(client.repository.get_model_url(saved_model_details))
model_uid = client.repository.get_model_uid(saved_model_details)
print("Saved model uid: " + model_uid)

In [ ]:
deployment_details = client.deployments.create(model_uid, "My object detection deployment")
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

In [ ]:
# client.training.delete(training_run_guid_async)
# client.repository.delete(definition_uid)
# client.deployments.delete(deployments_uid)